# Note:

The below approaches are performed on 15 stories for demonstration purposes. Please note that this is scalable to the entire dataset as well. Full analysis is omitted due to computational limitations.

# Approach 1
1. Create a list of the top ten proper nouns from each story.
2. To identify the people on the list, see if any of these terms are followed by a verb (40 percent).
(It's most likely a person)
3. Words that are not followed by a verb at least 40% of the time are removed.

In [1]:
#run this cell only for colab
from google.colab import drive
drive.mount('/content/drive')
!cp /content/drive/MyDrive/plots ./plots
!cp /content/drive/MyDrive/titles ./titles


Mounted at /content/drive


In [2]:
!pip install TextBlob
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


In [15]:
# first 10 stories
stories = []
story = ""
with open('plots') as f:
    for line in f:
        if len(stories) == 16:
            break
        if '<EOS>' in line:
          stories.append(story)
          story = ""
        else:
          story = story + line
stories = [story.replace("\n", " ") for story in stories]
stories = [story.replace("\\", "") for story in stories]

In [16]:
with open('titles') as titles_file:
  titles = titles_file.readlines()
titles = [title.replace("\n", "") for title in titles]

In [17]:
import string
from textblob import TextBlob
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('brown')
nltk.download('punkt')

#clean up
# clearingSymbols = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~’‘'
# for c in clearingSymbols:
#     text = text.replace(c, '')

for index, story in enumerate(stories):
    blob = TextBlob(story)

    verbs = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
    a = blob.tags  #pos tagging

    # retruns the top 10 propernouns 
    def prn(plotblob):
        d = plotblob.np_counts # noun counter 
        d = dict(d)
        top = sorted(d, key=d.get, reverse=True)
        return top[0:10]

    dictionary = blob.np_counts
    # checks if the noun followed by verb 50% of the time 
    for word in prn(blob):
        count = 0
        for i in range(0, len(a)-1):
            if a[i][0] == word or a[i][0] == word.title():
                if a[i+1][1] in verbs:
                    count = count + 1
        if count / dictionary[word] > 0.4:
            protaganist = word 
            print("The protagonist of the story " + titles[index] + " is :-" + protaganist) 
            break


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
The protagonist of the story Animal Farm is :-napoleon
The protagonist of the story A Clockwork Orange (novel) is :-alex
The protagonist of the story The Plague is :-tarrou
The protagonist of the story Actaeon is :-actaeon
The protagonist of the story A Fire Upon the Deep is :-johanna
The protagonist of the story Anyone Can Whistle is :-fay
The protagonist of the story A Funny Thing Happened on the Way to the Forum is :-senex
The protagonist of the story Army of Darkness is :-ash
The protagonist of the story The Birth of a Nation is :-cameron
The protagonist of the story Blade Runner

# Limitation
This approach is not ideal because it appears to be failing in some edge cases. As an example, **"Apple purchased Mitsubishi stock, and Apple fans are overjoyed."** Apple is considered a noun here, which is correct, but in the context of the sentence, Apple is an organization. As a result, I have discarded this approach.

In [11]:
stories[10]

'In Los Angeles in November 2019, ex-police officer Rick Deckard is detained by officer Gaff and brought to his former supervisor, Bryant.Deckard, whose job as a "Blade Runner" was to track down bioengineered beings known as replicants and "retire" (a euphemism for killing) them, is informed that four have come to Earth illegally.As Tyrell Corporation Nexus-6 models, they have only a four-year lifespan and may have come to Earth to try to extend their lives.Deckard watches a video of a Blade Runner named Holden administering the "Voight-Kampff" test designed to distinguish replicants from humans based on their emotional response to questions.The test subject, Leon, shoots Holden after Holden asks about Leon\'s mother.Bryant wants Deckard to retire Leon and the other three replicants: Roy Batty, Zhora, and Pris.Deckard initially refuses, but after Bryant ambiguously threatens him, he reluctantly agrees.Deckard begins his investigation at the Tyrell Corporation to ensure that the test wo

# Approach 2

This the prefered approach. It consists of: 
1. **NLTK NER classifer**, which contains a person classifier as well 
2. The protaganist is extracted by calculating the **frequency of the person occurences** in each story.
3. **Coreference Resolution** is performed for enriching the frequency information and also to identfy the gender of the protaganist 
4. Finally, in the case where coreference resolution fails in gender identification, a **naive bayes classifer** trained on gender name data is employed to classify the protaganist gender.

In [ ]:
!pip install flair
!pip install torch==1.9.0
!pip install allennlp==2.1.0 allennlp-models==2.1.0
!pip install transformers==4.1.1


In [18]:
import nltk
from nltk import pos_tag, word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from tqdm import tqdm
import re
import string
from itertools import combinations
from collections import Counter
from flair.models import SequenceTagger
from flair.data import Sentence
from nltk.corpus import names
from nltk import NaiveBayesClassifier as NBC
from nltk import classify
import random
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('gutenberg')
nltk.download('names')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Unzipping corpora/names.zip.


True

In [19]:
#training gender classifier 

def gender_features(word):
    return {'last_letter': word[-1]}


maleNames = [(name, 'male') for name in names.words('male.txt')]
femaleNames = [(name, 'female') for name in names.words('female.txt')]
allNames = maleNames + femaleNames
random.shuffle(allNames)
featureData = [(gender_features(namelist), gender)for (namelist, gender) in allNames]
test_data = featureData[:800]
train_data = featureData[800:]
classifier = NBC.train(train_data)

In [20]:
# Use flair named entity recognition
tagger = SequenceTagger.load('ner')

2021-10-13 18:15:41,838 --------------------------------------------------------------------------------
2021-10-13 18:15:41,840 The model key 'ner' now maps to 'https://huggingface.co/flair/ner-english' on the HuggingFace ModelHub
2021-10-13 18:15:41,842  - The most current version of the model is automatically downloaded from there.
2021-10-13 18:15:41,843  - (you can alternatively manually download the original model at https://nlp.informatik.hu-berlin.de/resources/models/ner/en-ner-conll03-v0.4.pt)
2021-10-13 18:15:41,844 --------------------------------------------------------------------------------


Downloading:   0%|          | 0.00/432M [00:00<?, ?B/s]

2021-10-13 18:15:53,337 loading file /root/.flair/models/ner-english/4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4


In [21]:
#extracting all the persons 
all_story_names = []
for s in stories:
  names = []
  sent = nltk.tokenize.sent_tokenize(s)
  # Get all the names of entities tagged as people
  for line in tqdm(sent):
    sentence = Sentence(line)
    tagger.predict(sentence)
    for entity in sentence.to_dict(tag_type='ner')['entities']:
      if entity['labels'][0].value == 'PER':
        names.append(entity['text'])
  all_story_names.append(names)

100%|██████████| 35/35 [01:02<00:00,  1.78s/it]


In [ ]:
all_story_names[0]

In [23]:
# Remove any punctuation within the names

protagonists = []

for story_names in all_story_names:
  names = []
  for name in story_names:
    names.append(name.translate(str.maketrans('', '', string.punctuation)))
  
  result = [item for items, c in Counter(story_names).most_common()
                                        for item in [items] * c]
  #extracting the protagonist by calculating the frequency
  protagonist =  Counter(names).most_common()[0][0]
  protagonists.append(protagonist)
  # print(Counter(names).most_common()) 



In [24]:
protagonists

['Napoleon',
 'Alex',
 'Rieux',
 'Artemis',
 'Pham',
 'Paul',
 'Fay',
 'Pseudolus',
 'Ash',
 'Cameron',
 'Deckard',
 'Bart',
 'Jeffrey',
 'Deckard',
 'Barry',
 'Buffy']

### Gender identification using coreference resolution and Gender classifier

In [ ]:
from allennlp.predictors.predictor import Predictor
model_url = 'https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2021.03.10.tar.gz'
predictor = Predictor.from_path(model_url)

In [26]:
for index, protagonist in enumerate(protagonists):
  pred = predictor.predict(
      document=stories[index]
  )
  clusters = pred['clusters']
  document = pred['document']

  n = 0
  doc = {}
  for obj in document:
      doc.update({n:  obj})
      n = n+1
# captures all the coreferenc clusters of a particular noun
  clus_all = []
  cluster = []
  clus_one = {}
  for i in range(0, len(clusters)):
      one_cl = clusters[i]
      for count in range(0, len(one_cl)):
          obj = one_cl[count]
          for num in range((obj[0]), (obj[1]+1)):
              for n in doc:
                  if num == n:
                      cluster.append(doc[n])
      clus_all.append(cluster)
      cluster = []
  # print(clus_all)  # And finally, this shows all coreferences


  gender = "na"


  for cluster in clus_all:
    if protagonist in cluster:
      cluster_lower = [word.lower() for word in cluster]
      if "she" in cluster_lower or "her" in cluster_lower or "hers" in cluster_lower:
        gender = "female"
      if "he" in cluster_lower or "him" in cluster_lower or "his" in cluster_lower:
        gender = "male"
  
  if gender == "na":
    gender = classifier.classify(gender_features(protagonist))
  
  print("gender of protagonist " + protagonist + " of the story " + titles[index] + " is " + gender)




gender of protagonist Napoleon of the story Animal Farm is male
gender of protagonist Alex of the story A Clockwork Orange (novel) is male
gender of protagonist Rieux of the story The Plague is male
gender of protagonist Artemis of the story Actaeon is female
gender of protagonist Pham of the story A Fire Upon the Deep is male
gender of protagonist Paul of the story All Quiet on the Western Front is male
gender of protagonist Fay of the story Anyone Can Whistle is female
gender of protagonist Pseudolus of the story A Funny Thing Happened on the Way to the Forum is male
gender of protagonist Ash of the story Army of Darkness is male
gender of protagonist Cameron of the story The Birth of a Nation is male
gender of protagonist Deckard of the story Blade Runner is male
gender of protagonist Bart of the story Blazing Saddles is male
gender of protagonist Jeffrey of the story Blue Velvet (film) is male
gender of protagonist Deckard of the story Blade Runner 2: The Edge of Human is male
gend

#Improvements

1. In some case antagonist might have higest frequency. A way to elminate this would be to calculate average sentiment of the sentences containing the top candidates for protagonists and selecting those with a net positive sentiment.
2. The case of multiple protagonists in a story could be figured out by identifying the distribution of the occurences of the characters. If the distribution seems to be even, then there is a possibility of multiple or no protagonists. 
3. Creation of an external ground truth can be achieved by utilising search engine APIs, by converting the title to a question (for eg. "Who is the protagonist of Buffy the Vampire Slayer?") and then extracting the answer from the search engine results. In the case that search engine querying is expensive, the open source movie database TMDB can be used to capture the protagonist as well as their gender. 
